In [1]:
import sys

sys.path.append("..")
from function.retriever import initialize_retriever

retriever = initialize_retriever()

metadatas = [entry.metadata for entry in retriever.vectorstore.docstore._dict.values()]
all_stored_venues = set(_["company"] for _ in metadatas)
all_stored_venues

Loaded existing FAISS index from faiss_db


{'Eden Gardens Moorpark'}

In [2]:
from pathlib import Path

from function.cloud import list_files

all_venues = set([Path(path).stem for path in list_files(filter=r"venues/")])

unprocessed_venues = all_venues - all_stored_venues
venue = list(unprocessed_venues)[1]
venue


'Serendipity Garden'

In [6]:
from function.retriever import preprocess_document
from dotenv import load_dotenv

load_dotenv(override=True)

info = preprocess_document("Majestic Garden Hotel")


searching for Majestic Garden Hotel.pdf on google cloud...
downloading Majestic Garden Hotel.pdf from google cloud...
sending Majestic Garden Hotel.pdf to Adobe...
extracting text from pdf...
2024-12-21 23:53:20 unzip file
2024-12-21 23:53:20 open json file
2024-12-21 23:53:20 extract text
generating image descriptions for Majestic Garden Hotel.pdf
Generating image descriptions...
   (1/2) /var/folders/g6/gqdf2y4n3jj4f0vml_0s_q1r0000gn/T/tmpf14rvmsh/figures/fileoutpart1.png
   (2/2) /var/folders/g6/gqdf2y4n3jj4f0vml_0s_q1r0000gn/T/tmpf14rvmsh/figures/fileoutpart0.png
skipping /var/folders/g6/gqdf2y4n3jj4f0vml_0s_q1r0000gn/T/tmpf14rvmsh/figures/fileoutpart0.png
uploading adobe_extracted_directory to google cloud


In [3]:
from function.retriever import preprocess_document
from dotenv import load_dotenv

load_dotenv(override=True)

info = preprocess_document(venue)


searching for Serendipity Garden.pdf on google cloud...
downloading Serendipity Garden.pdf from google cloud...
sending Serendipity Garden.pdf to Adobe...
extracting text from pdf...
2024-12-21 23:50:56 unzip file
2024-12-21 23:50:56 open json file
2024-12-21 23:50:56 extract text
generating image descriptions for Serendipity Garden.pdf
Generating image descriptions...
   (1/9) /var/folders/g6/gqdf2y4n3jj4f0vml_0s_q1r0000gn/T/tmpghn7yo8h/figures/fileoutpart8.png
   (2/9) /var/folders/g6/gqdf2y4n3jj4f0vml_0s_q1r0000gn/T/tmpghn7yo8h/figures/fileoutpart7.png
   (3/9) /var/folders/g6/gqdf2y4n3jj4f0vml_0s_q1r0000gn/T/tmpghn7yo8h/figures/fileoutpart6.png
   (4/9) /var/folders/g6/gqdf2y4n3jj4f0vml_0s_q1r0000gn/T/tmpghn7yo8h/figures/fileoutpart4.png
skipping /var/folders/g6/gqdf2y4n3jj4f0vml_0s_q1r0000gn/T/tmpghn7yo8h/figures/fileoutpart4.png
   (5/9) /var/folders/g6/gqdf2y4n3jj4f0vml_0s_q1r0000gn/T/tmpghn7yo8h/figures/fileoutpart5.png
   (6/9) /var/folders/g6/gqdf2y4n3jj4f0vml_0s_q1r0000gn/T/

In [4]:
import os

from function.retriever import add_documents_to_retriever

PERSIST_DIRECTORY: str = os.getenv("DATABASE_DIR")

add_documents_to_retriever({venue: info}, retriever)
retriever.vectorstore.save_local(PERSIST_DIRECTORY)


Processed document: Serendipity Garden


In [5]:
info

{'doc_id': 'f7a7743d-5aa4-4e92-bcea-037411024424',
 'text_content': [{'index': 0,
   'text': 'T\nh\ne\n \nW\nh\no\nl\ne\n \nP\na\nc\nk\na\ng\ne\n \n\n\nO\nu\nr\n \nu\nb\ne\nr\n \nc\no\nm\np\nl\ne\nt\ne\n \np\na\nc\nk\na\ng\ne\n \ni\ns\n \nw\nh\na\nt\n \ns\ne\nt\ns\n \nu\np\n \na\np\na\nr\nt\n \nf\nr\no\nm\n \na\nl\nl\n \no\nt\nh\ne\nr\n \nv\ne\nn\nu\ne\ns\n.\n \nW\ne\n \nd\no\n \ni\nt\n \na\nl\nl\n,\n \ns\no\n \ny\no\nu\n \na\nn\nd\n \ny\no\nu\nr\n \nl\no\nv\ne\nd\n \no\nn\ne\ns\n \nc\na\nn\n \nr\ne\nl\na\nx\n \na\nn\nd\n \ne\nn\nj\no\ny\n \ny\no\nu\nr\n \nm\na\ng\ni\nc\na\nl\n \nd\na\ny\n!\n \n\n\nB\nO\nO\nK\n \nA\n \nT\nO\nU\nR\n '},
  {'index': 1,
   'text': 'O\nP\nE\nN\n \nH\nO\nU\nS\nE\n \nE\nV\nE\nN\nT\n \n\n\nT\nh\ni\ns\n \ne\nv\ne\nn\nt\n \ni\ns\n \nh\ne\nl\nd\n \na\nt\n \nt\nh\ne\n \nb\ne\ng\ni\nn\nn\ni\nn\ng\n \no\nf\n \ne\na\nc\nh\n \ns\ne\na\ns\no\nn\n \nj\nu\ns\nt\n \nf\no\nr\n \no\nu\nr\n \nB\nO\nO\nK\nE\nD\n \nc\no\nu\np\nl\ne\ns\n \nw\ni\nt\nh\n \nw\ne\nd\nd\ni\nn\ng\ns

In [5]:
retriever.vectorstore.docstore._dict

{'7c00f644-3585-483f-9c0b-66d2829924eb': Document(metadata={'content_id': 'f454090d-c8e4-43bc-a591-d3f00292af2b_text_0', 'doc_id': 'f454090d-c8e4-43bc-a591-d3f00292af2b', 'company': 'Eden Gardens Moorpark', 'type': 'text'}, page_content='2\n0\n2\n5\n \nW\ne\nd\nd\ni\nn\ng\n \nS\ne\na\ns\no\nn\n '),
 'a1617bc7-fc6b-426f-a820-d23f4870ab4c': Document(metadata={'content_id': 'f454090d-c8e4-43bc-a591-d3f00292af2b_text_1', 'doc_id': 'f454090d-c8e4-43bc-a591-d3f00292af2b', 'company': 'Eden Gardens Moorpark', 'type': 'text'}, page_content='$\n2\n2\n5\n0\n \nw\ne\ne\nk\ne\nn\nd\n \n(\nF\nr\ni\nd\na\ny\n \n–\n \nS\nu\nn\nd\na\ny\n)\n '),
 'be3afcdf-d11c-4be5-bbff-5135f0bc28c4': Document(metadata={'content_id': 'f454090d-c8e4-43bc-a591-d3f00292af2b_text_2', 'doc_id': 'f454090d-c8e4-43bc-a591-d3f00292af2b', 'company': 'Eden Gardens Moorpark', 'type': 'text'}, page_content='W\nh\na\nt\n \nI\ns\n \nI\nn\nc\nl\nu\nd\ne\nd\n \nw\ni\nt\nh\n \ny\no\nu\nr\n \nM\no\nn\nt\nh\n \no\nf\n \nC\no\no\nr\nd\ni\n